# SVG Repair Colab Demo\nRuntime setup + model downloads (SAM2.1-B, YOLO26, GroundingDINO, Big-LaMa) + UI launch.

In [1]:
!nvidia-smi || true
import torch
print('torch.cuda.is_available =', torch.cuda.is_available())


Sun Feb  8 03:28:47 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
from pathlib import Path

REPO_DIR = Path('/content/svg-colab')
if not REPO_DIR.exists():
    print('Cloning the repository...')
    !git clone https://github.com/LIdro/svg-colab.git {REPO_DIR}
    print('Expected path:', REPO_DIR)

%cd /content/svg-colab
!sed -i '/pypotrace/d' colab_demo/services/requirements-colab.txt
!pip install -r colab_demo/services/requirements-colab.txt

Cloning the repository...
Cloning into '/content/svg-colab'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 6), reused 22 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 35.79 KiB | 5.96 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Expected path: /content/svg-colab
/content/svg-colab
  Cloning https://github.com/ultralytics/CLIP.git to /tmp/pip-req-build-kcrc74eo
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/CLIP.git /tmp/pip-req-build-kcrc74eo
  Resolved https://github.com/ultralytics/CLIP.git to commit 88ade288431a46233f1556d1e141901b3ef0a36b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.1 M

In [5]:
import os

# Read secrets from Colab Secrets panel if available:
# - OPENROUTER_API_KEY
# - HF_TOKEN (optional)
# - SAM2_URL (optional direct URL override for sam2.1_b.pt)
# - BIG_LAMA_URL (optional direct URL override for big-lama.pt)
try:
    from google.colab import userdata  # type: ignore
except Exception:
    userdata = None

def read_secret(name: str):
    if userdata is None:
        return os.environ.get(name)
    try:
        return userdata.get(name)
    except Exception:
        return os.environ.get(name)

OPENROUTER_API_KEY = read_secret('OPENROUTER_API_KEY')
HF_TOKEN = read_secret('HF_TOKEN') or read_secret('HUGGINGFACE_TOKEN')
SAM2_URL = read_secret('SAM2_URL')
BIG_LAMA_URL = read_secret('BIG_LAMA_URL')

if OPENROUTER_API_KEY:
    os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
if SAM2_URL:
    os.environ['SAM2_URL'] = SAM2_URL
if BIG_LAMA_URL:
    os.environ['BIG_LAMA_URL'] = BIG_LAMA_URL

print('OPENROUTER_API_KEY loaded:', bool(OPENROUTER_API_KEY))
print('HF_TOKEN loaded:', bool(HF_TOKEN))
print('SAM2_URL provided:', bool(SAM2_URL))
print('BIG_LAMA_URL provided:', bool(BIG_LAMA_URL))


OPENROUTER_API_KEY loaded: True
HF_TOKEN loaded: False
SAM2_URL provided: False
BIG_LAMA_URL provided: False


In [6]:
import os
import shutil
from pathlib import Path
import requests
from huggingface_hub import hf_hub_download

ROOT = Path('/content/svg-colab')
WEIGHTS_DIR = ROOT / '.colab_models'
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

def ensure_http_download(url: str, dst: Path):
    if dst.exists() and dst.stat().st_size > 0:
        print('[skip] exists:', dst)
        return dst
    print('[download]', url, '->', dst)
    with requests.get(url, stream=True, timeout=120) as r:
        r.raise_for_status()
        with open(dst, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
    return dst

def try_hf(repo_id: str, filename: str, out_name: str, token=None):
    out_path = WEIGHTS_DIR / out_name
    if out_path.exists() and out_path.stat().st_size > 0:
        print('[skip] exists:', out_path)
        return out_path
    try:
        p = hf_hub_download(repo_id=repo_id, filename=filename, token=token, local_dir=str(WEIGHTS_DIR))
        shutil.copy2(p, out_path)
        print('[ok] HF:', repo_id, filename, '->', out_path)
        return out_path
    except Exception as e:
        print('[warn] HF download failed:', repo_id, filename, e)
        return None

# YOLO26 weights
YOLO26L = try_hf('openvision/yoloe-26l-seg', 'model.pt', 'yoloe-26l-seg.pt', token=HF_TOKEN)
YOLO26X = try_hf('openvision/yoloe-26x-seg', 'model.pt', 'yoloe-26x-seg.pt', token=HF_TOKEN)

# SAM2.1-B weight (same default used in svg-repair: sam2.1_b.pt)
SAM2 = WEIGHTS_DIR / 'sam2.1_b.pt'
if not SAM2.exists() and os.environ.get('SAM2_URL'):
    try:
        ensure_http_download(os.environ['SAM2_URL'], SAM2)
    except Exception as e:
        print('[warn] SAM2_URL download failed:', e)

if not SAM2.exists():
    local_repo_sam = ROOT / 'svg-repair' / 'sam2.1_b.pt'
    if local_repo_sam.exists():
        shutil.copy2(local_repo_sam, SAM2)
        print('[ok] copied SAM2 from repo file:', local_repo_sam)

if not SAM2.exists():
    # Best-effort public URLs (may change; SAM2_URL secret is most reliable)
    sam_urls = [
        'https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2.1_b.pt',
        'https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2_b.pt',
    ]
    for url in sam_urls:
        try:
            ensure_http_download(url, SAM2)
            break
        except Exception as e:
            print('[warn] SAM2 URL failed:', url, e)

# GroundingDINO config + weights
GDINO_CFG = WEIGHTS_DIR / 'groundingdino_swint_ogc.cfg.py'
GDINO_PTH = WEIGHTS_DIR / 'groundingdino_swint_ogc.pth'
ensure_http_download('https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinT_OGC.py', GDINO_CFG)
ensure_http_download('https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swint_ogc.pth', GDINO_PTH)

# Big-LaMa checkpoint (tries direct URL override first, then HF candidates)
BIG_LAMA = WEIGHTS_DIR / 'big-lama.pt'
if not BIG_LAMA.exists() and os.environ.get('BIG_LAMA_URL'):
    try:
        ensure_http_download(os.environ['BIG_LAMA_URL'], BIG_LAMA)
    except Exception as e:
        print('[warn] BIG_LAMA_URL download failed:', e)

if not BIG_LAMA.exists():
    for repo_id, fname in [
        ('fashn-ai/lama', 'big-lama.pt'),
        ('CodeGoat24/Big-Lama', 'big-lama.pt'),
    ]:
        p = try_hf(repo_id, fname, 'big-lama.pt', token=HF_TOKEN)
        if p and p.exists():
            BIG_LAMA = p
            break

# Export runtime env vars used by the Colab API
if Path(SAM2).exists():
    os.environ['SAM_WEIGHTS'] = str(SAM2)
if YOLO26L and Path(YOLO26L).exists():
    os.environ['YOLO26L_WEIGHTS'] = str(YOLO26L)
if YOLO26X and Path(YOLO26X).exists():
    os.environ['YOLO26X_WEIGHTS'] = str(YOLO26X)
os.environ['GDINO_CONFIG'] = str(GDINO_CFG)
os.environ['GDINO_WEIGHTS'] = str(GDINO_PTH)

if Path(BIG_LAMA).exists():
    os.environ['BIG_LAMA_CHECKPOINT'] = str(BIG_LAMA)
    os.environ['BIG_LAMA_COMMAND'] = 'python /content/svg-colab/colab_demo/services/inpaint_big_lama.py --image {image} --mask {mask} --output {output}'

# Optional Qualcomm LaMa command (weights handled by qai_hub_models at runtime)
os.environ['QUALCOMM_LAMA_DILATED_COMMAND'] = 'python /content/svg-colab/colab_demo/services/inpaint_qualcomm_lama_dilated.py --image {image} --mask {mask} --output {output}'

print('SAM_WEIGHTS =', os.environ.get('SAM_WEIGHTS'))
print('YOLO26L_WEIGHTS =', os.environ.get('YOLO26L_WEIGHTS'))
print('YOLO26X_WEIGHTS =', os.environ.get('YOLO26X_WEIGHTS'))
print('GDINO_CONFIG =', os.environ.get('GDINO_CONFIG'))
print('GDINO_WEIGHTS =', os.environ.get('GDINO_WEIGHTS'))
print('BIG_LAMA_CHECKPOINT exists =', Path(os.environ.get('BIG_LAMA_CHECKPOINT', '/tmp/missing')).exists())
print('OPENROUTER_API_KEY loaded =', bool(os.environ.get('OPENROUTER_API_KEY')))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.pt:   0%|          | 0.00/78.9M [00:00<?, ?B/s]

[ok] HF: openvision/yoloe-26l-seg model.pt -> /content/svg-colab/.colab_models/yoloe-26l-seg.pt


model.pt:   0%|          | 0.00/171M [00:00<?, ?B/s]

[ok] HF: openvision/yoloe-26x-seg model.pt -> /content/svg-colab/.colab_models/yoloe-26x-seg.pt
[download] https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2.1_b.pt -> /content/svg-colab/.colab_models/sam2.1_b.pt
[download] https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinT_OGC.py -> /content/svg-colab/.colab_models/groundingdino_swint_ogc.cfg.py
[download] https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swint_ogc.pth -> /content/svg-colab/.colab_models/groundingdino_swint_ogc.pth


HTTPError: 404 Client Error: Not Found for url: https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swint_ogc.pth

In [ ]:
%cd /content/svg-colab
!python -m uvicorn colab_demo.services.api:app --host 0.0.0.0 --port 5700


In [ ]:
# Run in another Colab tab/session after API is running
%cd /content/svg-colab
!python colab_demo/ui/gradio_app.py
